
================================================================================

PROJET DATA SCIENCE - PRÉDICTION DU CHURN TÉLÉCOM

================================================================================







Étape 1 : Choix du problème


================================================================================

CONTEXTE :
Une entreprise de télécommunications fait face à un taux d'attrition élevé.
L'objectif est d'identifier les clients à risque pour mettre en place des actions de rétention ciblées.

OBJECTIFS :
- Atteindre une précision de prédiction d'au moins 80%
- Identifier correctement 75% des clients qui vont partir
- Réduire le taux d'attrition de 15%

================================================================================

============================================================================

Étape 2 : Exploration des données

============================================================================




import des libraries

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
import warnings

warnings.filterwarnings('ignore')

# Configuration de l'affichage
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")
pd.set_option('display.max_columns', None)

print("✅ Bibliothèques importées avec succès!")


✅ Bibliothèques importées avec succès!



CHARGEMENT DES DONNÉES


In [3]:
try:
    data = pd.read_csv('WA_Fn-UseC_-Telco-Customer-Churn.csv')
    print("✅ Données chargées avec succès!")
except FileNotFoundError:
    print("❌ ERREUR : Fichier non trouvé!")
    print("\n📋 INSTRUCTIONS :")
    print("1. Allez sur : https://www.kaggle.com/datasets/blastchar/telco-customer-churn")
    print("2. Téléchargez le fichier CSV")
    print("3. Placez-le dans le même dossier que ce script")
    print("4. Relancez le script")
    exit()

✅ Données chargées avec succès!


EXPLORATION INITIALE


In [5]:
print("\n📊 DIMENSIONS DU DATASET :")
print(f"   Nombre de lignes (clients) : {data.shape[0]}")
print(f"   Nombre de colonnes (variables) : {data.shape[1]}")

print("\n📋 APERÇU DES PREMIÈRES LIGNES :")
print(data.head())

print("\n🔍 INFORMATIONS SUR LES TYPES DE DONNÉES :")
print(data.info())

print("\n📈 STATISTIQUES DESCRIPTIVES (variables numériques) :")
print(data.describe())

print("\n🎯 DISTRIBUTION DE LA VARIABLE CIBLE (Churn) :")
churn_counts = data['Churn'].value_counts()
churn_percentage = data['Churn'].value_counts(normalize=True) * 100
print(f"   Non : {churn_counts['No']} clients ({churn_percentage['No']:.1f}%)")
print(f"   Oui : {churn_counts['Yes']} clients ({churn_percentage['Yes']:.1f}%)")


📊 DIMENSIONS DU DATASET :
   Nombre de lignes (clients) : 7043
   Nombre de colonnes (variables) : 21

📋 APERÇU DES PREMIÈRES LIGNES :
   customerID  gender  SeniorCitizen Partner Dependents  tenure PhoneService  \
0  7590-VHVEG  Female              0     Yes         No       1           No   
1  5575-GNVDE    Male              0      No         No      34          Yes   
2  3668-QPYBK    Male              0      No         No       2          Yes   
3  7795-CFOCW    Male              0      No         No      45           No   
4  9237-HQITU  Female              0      No         No       2          Yes   

      MultipleLines InternetService OnlineSecurity OnlineBackup  \
0  No phone service             DSL             No          Yes   
1                No             DSL            Yes           No   
2                No             DSL            Yes          Yes   
3  No phone service             DSL            Yes           No   
4                No     Fiber optic             